In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cerebras/btlm-3b-8k-base", device="cuda")
model = AutoModelForCausalLM.from_pretrained("cerebras/btlm-3b-8k-base", trust_remote_code=True, torch_dtype="auto", device="cuda")

In [ ]:
prompt = """SYSTEM: You are an agent tasked with creating better, more descriptive image prompts that result in higher quality outputs. You are given a user prompt, and determine how, if it all, to change the users image. Do you understand?
AGENT: Yes, I understand.
SYSTEM: Great. Below there will be a set of examples. For each example, you will be given the users prompt and the expected output. You will be then given a new user prompt and have to generate a new output. Do you understand?
AGENT: Yes.

PROMPT: A picture of a cat.
AGENT: masterpiece:2, Profession Photograph, A cat sitting on a chair, Orange, White whiskers

PROMPT: The beach
AGENT: masterpiece:3, A beautiful beach, Palm trees, blue sky, white clouds, clear sky

PROMPT: An anime girl
AGENT:"""
# Change the last prompt to whatever you want to test
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:
# Generate text using the model
outputs = model.generate(
    **inputs,
    num_beams=5,
    max_new_tokens=50,
    early_stopping=True,
    no_repeat_ngram_size=2
)
# Convert the generated token IDs back to text
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text)